## 5. Optimización de inventarios y políticas de reabastecimiento


### Importar Datos

In [76]:
# Importar Datos
import pandas as pd
import numpy as np
import math
df_prediccion  = pd.read_csv('../data/data_predicción.csv')
df_inventario  = pd.read_excel('../data/Inventario_actual.xlsx')
df_lt  = pd.read_excel('../data/Lead_time_de_cada_producto.xlsx')
df_pedidos_confirmados = pd.read_excel('../data/Pedidos_de_venta_confirmados.xlsx')
df_compras_programadas  = pd.read_excel('../data/Ordenes_de_compra_programadas.xlsx')

# Pivotar el DataFrame
df_compras_programadas = df_compras_programadas.pivot_table(index=['Año', 'Mes', 'Fecha de entrega'], columns='Producto', values='Cantidad', aggfunc='sum', fill_value=0)
# Resetear el índice para hacer más legible
df_compras_programadas=df_compras_programadas.reset_index()
meses_dict = {
    "Enero": 1, "Febrero": 2, "Marzo": 3, "Abril": 4,
    "Mayo": 5, "Junio": 6, "Julio": 7, "Agosto": 8,
    "Septiembre": 9, "Octubre": 10, "Noviembre": 11, "Diciembre": 12
}
# Usar el diccionario para mapear los valores de la columna 'Mes'
df_compras_programadas['Mes'] = df_compras_programadas['Mes'].map(meses_dict)

meses_ordenados = [1,2,3,4,5,6,7,8,9,10,11,12]
df_meses = pd.DataFrame({"Año": [2024] * 12, "Mes": meses_ordenados})
df_completo = df_meses.merge(df_compras_programadas, on=["Año", "Mes"], how="left")
df_completo["Mes"] = pd.Categorical(df_completo["Mes"], categories=meses_ordenados, ordered=True)
df_compras_programadas = df_completo.sort_values(["Año", "Mes"]).reset_index(drop=True)
# Agrupar por meses
df_compras_programadas =df_compras_programadas.groupby(['Año', 'Mes']).sum(numeric_only = True).reset_index()
df_compras_programadas

df_prediccion


C:\Users\user\AppData\Local\Temp\ipykernel_18048\3632074783.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_compras_programadas =df_compras_programadas.groupby(['Año', 'Mes']).sum(numeric_only = True).reset_index()


,Unnamed: 0,año,mes,Predicciones_PA,Predicciones_PB,Predicciones_PC,Predicciones_PD
0,0,2024,1,465.0,534.0,409.0,376.0
1,1,2024,2,470.0,540.0,414.0,380.0
2,2,2024,3,475.0,546.0,419.0,384.0
3,3,2024,4,480.0,552.0,423.0,389.0
4,4,2024,5,485.0,557.0,428.0,393.0
5,5,2024,6,490.0,563.0,433.0,397.0
6,6,2024,7,495.0,569.0,437.0,402.0
7,7,2024,8,501.0,575.0,442.0,406.0
8,8,2024,9,506.0,581.0,447.0,410.0
9,9,2024,10,511.0,587.0,451.0,414.0


### Cálculo del Inventario Óptimo

Para evitar roturas de stock y excesos de inventario, calculamos el **Stock de Seguridad (SS)** y el **Punto de Reorden (ROP)**:

SS 
$$ SS = Z_{NS} \times \sigma_d \times \sqrt{T} $$

- **SS**: Stock de seguridad, inventario extra para evitar quiebres.  
- **Z_NS**: Factor Z según el nivel de servicio deseado.  
- **σ_d**: Desviación estándar de la demanda diaria.  
- **T**: Tiempo de entrega (Lead Time) en días.

ROP

$$ ROP = (\text{Demanda proyectada en Lead Time}) + SS $$



Cálculo de z para un nivel de confianza de  95%

In [68]:

from scipy.stats import norm
# Nivel de confianza del 95%
confianza = 0.95
# Cálculo del valor Z
valor_z = norm.ppf(confianza)

print(f"El valor Z para un intervalo de confianza del 95% es: {valor_z:.4f}")

El valor Z para un intervalo de confianza del 95% es: 1.6449


Cálculo de σ_d (Desviación estándar de la demanda diaria)

In [69]:
# Calculamos la desviación estandar de la demanda mensual del pronóstico realizado
productos = ['Predicciones_PA', 'Predicciones_PB','Predicciones_PC','Predicciones_PD']
dic_std_productos = {}
for producto in productos:
    d_std_mensual = df_prediccion[producto].std()
    días_mes = 30
    d_std_diaria = d_std_mensual / np.sqrt(d_std_mensual)
    dic_std_productos[producto] = d_std_diaria

dic_std_productos


{'Predicciones_PA': np.float64(6.0127224106052255),
 'Predicciones_PB': np.float64(6.44149025078058),
 'Predicciones_PC': np.float64(5.708853623679093),
 'Predicciones_PD': np.float64(5.4549964947031215)}

Extracción de Lead Time de productos en dicccionario

In [70]:

lt_p = df_lt.set_index('Producto')['LT (meses)'].to_dict()
lt_p 

{'Producto A': 2, 'Producto B': 3, 'Producto C': 2, 'Producto D': 4}

 > Cálculo de stock de seguridad (SS)

In [71]:
def SS_value(valorz, lt_productos, desv_std_productos):
        dict_SS_productos = {}
        for key1 in lt_productos.keys() :
                for key2 in desv_std_productos.keys():
                        SS = valorz*desv_std_productos[key2]*np.sqrt(lt_productos [key1])
                        # Redondear hacia el mayor entero
                        dict_SS_productos[key1] = math.ceil(SS)    

        df_SS = pd.DataFrame(list(dict_SS_productos.items()), columns= ["Producto", "Stock_Seguridad"]) 

        return df_SS

df_stock_seguridad = SS_value(valor_z,lt_p,dic_std_productos )

df_stock_seguridad


,Producto,Stock_Seguridad
0,Producto A,13
1,Producto B,16
2,Producto C,13
3,Producto D,18


In [72]:
    
extraer_inventario_actual = df_inventario.set_index('Producto')['Cantidad'].to_dict()
inventario_actual_dict = extraer_inventario_actual.copy()


inventario_actual_dict

{'Producto A': 500, 'Producto B': 700, 'Producto C': 600, 'Producto D': 800}

In [73]:
demanda_total_anual = df_prediccion[['Predicciones_PA', 'Predicciones_PB', 'Predicciones_PC', 'Predicciones_PD']].sum()
dd =float(demanda_total_anual['Predicciones_PA']/365)
dd

34.43013698630137

In [74]:
# Crear DataFrame para almacenar los resultados
df_resultados = pd.DataFrame(columns=['Año', 'Mes', 'Producto', 'Demanda_Proyectada', 'Inventario_Necesario', 
                                      'Inventario_Inicial','Pedido_Solicitado', 'Pedido_Recibido', 'Inventario_Final', 'Reorder_Point'])

# Extraer el inventario inicial en un diccionario
inventario_actual_dict = df_inventario.set_index('Producto')['Cantidad'].to_dict()

# Cálculo de las demandas totales pronosticadas
demanda_total_anual = df_prediccion[['Predicciones_PA', 'Predicciones_PB', 'Predicciones_PC', 'Predicciones_PD']].sum()

# Definir los costos de pedido y de mantenimiento
costo_por_pedido = 1000  # Costo por hacer un pedido (S)
costo_por_unidad = 3  # Costo de mantener una unidad en inventario por año (H)

# Inicializar contador de meses de espera por reabastecimiento
contador_meses = {'Producto A':0, 'Producto B':0, 'Producto C':0, 'Producto D':0}
pedido_reabastecimiento = {'Producto A': 0, 'Producto B': 0, 'Producto C': 0, 'Producto D': 0}

# Iniciar el cálculo de inventarios y reabastecimiento
for idx, row in df_prediccion.iterrows():
    for producto in ['Producto A', 'Producto B', 'Producto C', 'Producto D']:
        
        # Extraer datos de demanda y lead time
        demanda_proyectada = row[f'Predicciones_P{producto[9:]}']
        demanda_anual_proyectada = demanda_total_anual[f'Predicciones_P{producto[9:]}']
        stock_seguridad = df_stock_seguridad.loc[df_stock_seguridad['Producto'] == producto, 'Stock_Seguridad'].values[0]
        lt = df_lt.loc[df_lt['Producto'] == producto, 'LT (meses)'].values[0]
        lt_dias = lt * 30  # Convertir lead time a días
        demanda_diaria = demanda_anual_proyectada / 365  # Demanda diaria
        # Calcular el reorder point
        reorder_point = round(demanda_diaria * lt_dias) + stock_seguridad  # Cálculo del reorder point
        #print(reorder_point)
        # Calcular el tamaño óptimo de pedido (Q)
        q = math.sqrt((2 * demanda_anual_proyectada * costo_por_pedido) / costo_por_unidad)


        # Extraer compras programadas para el mes actual
        if idx + 1 <= 12:
            compras_programadas = df_compras_programadas.loc[df_compras_programadas['Mes'] == idx + 1, producto].values[0]
        else:
            compras_programadas = 0
        # Manejo del inventario actual considerando el reabastecimiento después del lead time
        if contador_meses[producto] == lt:
            inventario_actual_dict[producto] =inventario_actual_dict[producto] + pedido_reabastecimiento[producto]  # Se suma el pedido recibido
            contador_meses[producto] = 0  # Reiniciar contador después del lead time
            pedido_recibido = pedido_reabastecimiento[producto]
        elif compras_programadas > 0 and contador_meses[producto] == lt :
            inventario_actual_dict[producto] =inventario_actual_dict[producto] + pedido_reabastecimiento[producto] + compras_programadas
            pedido_recibido = inventario_actual_dict[producto]

        elif compras_programadas > 0:
            pedido_recibido = compras_programadas
        else:
            pedido_recibido = 0
            # Si no se cumple el lead time, se incrementa el contador
            if contador_meses[producto]>0:
                contador_meses[producto] += 1 
            


        inventario_actual_producto = inventario_actual_dict[producto]

        # Calcular inventario final después de la demanda
        inventario_final = inventario_actual_producto - demanda_proyectada

        # Cálculo del inventario necesario
        inventario_necesario = inventario_actual_producto - demanda_proyectada
        if inventario_necesario >= 0:
            inventario_necesario = 0  # No es necesario más inventario
        else:
            inventario_necesario = abs(inventario_necesario)  # Se toma como cantidad faltante

        inventario_actual_dict[producto] = inventario_final
        # Determinar si se necesita un pedido de reabastecimiento
        
        if inventario_actual_dict[producto] <= float(reorder_point):
            
            if contador_meses[producto] == 0:  # No hay un pedido pendiente
                pedido_reabastecimiento[producto] = q + stock_seguridad
                pedido_solicitado = pedido_reabastecimiento[producto]
                contador_meses[producto] = 1  # Inicia el conteo del lead time
            else:
                pedido_solicitado = 0  
        else:
            pedido_solicitado = 0  # No se necesita pedido

        # Guardar resultados en el DataFrame
        resultado = {
            'Año': int(row['año']),
            'Mes': int(row['mes']),
            'Producto': producto,
            'Demanda_Proyectada': int(demanda_proyectada),
            'Inventario_Necesario': int(inventario_necesario),
            'Inventario_Inicial': int(inventario_actual_producto),
            'Pedido_Solicitado': int(pedido_solicitado),
            'Pedido_Recibido': int(pedido_recibido),
            'Inventario_Final': int(inventario_final),
            'Reorder_Point': int(reorder_point),
        }
        
        df_resultados = pd.concat([df_resultados, pd.DataFrame([resultado])], ignore_index=True)


#Guardar data
Pronóstico_Demanda_S_Inventarios = df_resultados.to_csv('../data/Pronóstico_Demanda_S_Inventarios.csv')

In [75]:
df_resultados

,Año,Mes,Producto,Demanda_Proyectada,Inventario_Necesario,Inventario_Inicial,Pedido_Solicitado,Pedido_Recibido,Inventario_Final,Reorder_Point
0,2024,1,Producto A,465,0,500,2907,0,35,2079
1,2024,1,Producto B,534,0,700,3118,0,166,3576
2,2024,1,Producto C,409,0,600,2733,0,191,1838
3,2024,1,Producto D,376,0,800,2624,0,424,3367
4,2024,2,Producto A,470,435,35,0,0,-435,2079
...,...,...,...,...,...,...,...,...,...,...
91,2025,11,Producto D,469,0,2050,2624,2624,1581,3367
92,2025,12,Producto A,582,0,3052,0,0,2470,2079
93,2025,12,Producto B,669,0,2524,0,0,1855,3576
94,2025,12,Producto C,516,0,3682,0,2733,3166,1838
